## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Punta Arenas,CL,-53.1500,-70.9167,51.91,54,75,23.02,broken clouds
1,1,Yellowknife,CA,62.4560,-114.3525,-25.58,88,75,3.44,broken clouds
2,2,Shingu,JP,33.7333,135.9833,43.92,62,30,4.41,scattered clouds
3,3,Caravelas,BR,-17.7125,-39.2481,75.92,90,93,7.76,overcast clouds
4,4,Krasnyy Yar,RU,46.5331,48.3456,31.96,92,83,15.77,broken clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [9]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
criteria_city_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
criteria_city_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Caravelas,BR,-17.7125,-39.2481,75.92,90,93,7.76,overcast clouds
6,6,Rikitea,PF,-23.1203,-134.9692,78.24,79,100,19.19,overcast clouds
10,10,Hilo,US,19.7297,-155.0900,80.33,71,75,6.91,broken clouds
15,15,Kahului,US,20.8947,-156.4700,79.97,66,0,5.01,clear sky
18,18,Mahebourg,MU,-20.4081,57.7000,76.24,100,75,3.44,broken clouds


In [10]:
# 4a. Determine if there are any empty rows.
criteria_city_df.count()

Unnamed: 0             240
City                   240
Country                240
Lat                    240
Lng                    240
Max Temp               240
Humidity               240
Cloudiness             240
Wind Speed             240
Current Description    240
dtype: int64

In [12]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
criteria_city_df = criteria_city_df.dropna()

In [14]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = criteria_city_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Caravelas,BR,75.92,overcast clouds,-17.7125,-39.2481,
6,Rikitea,PF,78.24,overcast clouds,-23.1203,-134.9692,
10,Hilo,US,80.33,broken clouds,19.7297,-155.0900,
15,Kahului,US,79.97,clear sky,20.8947,-156.4700,
18,Mahebourg,MU,76.24,broken clouds,-20.4081,57.7000,
24,Bredasdorp,ZA,70.92,overcast clouds,-34.5322,20.0403,
28,Contamana,PE,85.30,broken clouds,-7.3333,-75.0167,
32,Butaritari,KI,81.48,overcast clouds,3.0707,172.7902,
33,Savannah Bight,HN,80.65,clear sky,16.4500,-85.8500,
34,Bengkulu,ID,76.01,light rain,-3.8004,102.2655,


In [21]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat=row["Lat"]
    lng=row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
     hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
     print("Hotel not found....skipping.")
        

Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.


In [25]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(subset=['Hotel Name'])

In [26]:
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Caravelas,BR,75.92,overcast clouds,-17.7125,-39.2481,Pousada dos Navegantes
6,Rikitea,PF,78.24,overcast clouds,-23.1203,-134.9692,People ThankYou
10,Hilo,US,80.33,broken clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
15,Kahului,US,79.97,clear sky,20.8947,-156.4700,Maui Seaside Hotel
18,Mahebourg,MU,76.24,broken clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa


In [27]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [28]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [29]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))